# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['p', ':', 'n', 'I', '/']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([28, 24, 25, 40, 12,  7,  2, 19, 18, 42])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[28, 24, 25, 40, 12,  7,  2, 19],
        [24, 25, 40, 12,  7,  2, 19, 18],
        [25, 40, 12,  7,  2, 19, 18, 42],
        [40, 12,  7,  2, 19, 18, 42, 12],
        [12,  7,  2, 19, 18, 42, 12, 16]])
y training data
tensor([[24, 25, 40, 12,  7,  2, 19, 18],
        [25, 40, 12,  7,  2, 19, 18, 42],
        [40, 12,  7,  2, 19, 18, 42, 12],
        [12,  7,  2, 19, 18, 42, 12, 16],
        [ 7,  2, 19, 18, 42, 12, 16, 29]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[ 1.3714,  0.8524,  1.5221, -1.0615, -0.3488,  0.7318,  2.0752,
           0.4439],
         [ 0.3213,  0.5684,  1.0322, -1.5972, -1.5429,  1.0081,  0.8304,
           0.3172],
         [-1.5154,  0.3275,  0.8591,  0.2626,  1.4236,  0.4716, -0.7320,
           0.2738],
         [-0.0512, -1.6777, -1.0690, -0.1452,  1.2174,  0.5099, -0.1739,
           0.0405]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[ 0.9814, -0.0656,  0.0673,  0.5413,  1.1494, -1.3388,  0.2755,
           0.7445],
         [ 0.7848, -0.2118,  0.9142, -0.0479, -0.8624,  0.6424,  0.2042,
          -0.2691],
         [-0.0411, -0.9826,  0.0639,  0.2972, -0.8725, -1.1352,  0.3890,
           1.6370],
         [-0.8569,  1.3421,  0.2077, -0.0831, -1.7176,  0.6414, -0.1823,
          -0.8655],
         [ 1.2021, -0.4688, -0.5701, -1.3151,  0.4430,  0.5387,  0.0251,
           0.4245],
         [-1.1869, -1.4327, -0.0363,  1.5950, -0.4343, -2.0443,  1.6574,
          -0.3617],
         [-1.5154,  0.3275,  0.8591,  0.2626,  1.4236,  0.4716, -0.7320,
           0.2738],
         [ 0.3850,  0.5533, -0.3635, -1.2622, -1.1007, -0.7654,  0.2875,
           0.4457]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [23]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [48]:
d_k = d_q = d_v = 10

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False)


In [51]:
W_q.weight.shape

torch.Size([10, 8])

In [52]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [53]:
W_q.weight@x_embeddings_total[0][0]

tensor([-0.7016, -1.2739, -0.0510, -1.7870,  0.1321, -0.3128,  0.2754, -0.1269,
         0.8233, -0.8119], grad_fn=<MvBackward0>)

In [54]:
W_q(x_embeddings_total)

tensor([[[-7.0161e-01, -1.2739e+00, -5.1022e-02, -1.7870e+00,  1.3210e-01,
          -3.1276e-01,  2.7537e-01, -1.2691e-01,  8.2333e-01, -8.1188e-01],
         [ 6.0421e-01,  2.4767e-02, -7.9699e-01,  2.3967e-01, -3.9853e-01,
           7.0460e-01,  9.3921e-01,  1.2793e+00,  1.2973e+00,  9.4875e-01],
         [ 4.3131e-01, -5.0135e-02,  3.3971e-01, -2.6624e-02,  1.0610e+00,
          -5.5387e-01, -3.4110e-01, -4.1804e-01, -3.9566e-03,  7.0141e-01],
         [-1.1012e-01,  5.2924e-01,  5.2885e-01,  6.2617e-01,  4.3642e-01,
          -7.2202e-02, -7.8692e-01, -5.5516e-01, -1.5537e+00,  1.0767e-01],
         [-9.8368e-01,  2.0109e-01, -1.7731e-01, -3.4423e-01, -1.5610e+00,
           1.6362e+00,  3.5768e-01, -2.6799e-01, -3.1720e-01, -7.3949e-01],
         [-5.2773e-01,  2.4939e-01,  1.1789e+00, -3.0693e-02,  1.0144e+00,
          -1.8045e+00,  2.2515e-01,  3.8290e-01,  2.5219e-01,  1.7592e+00],
         [ 6.2366e-01,  2.8153e-01,  9.3578e-01,  7.8686e-01, -6.3206e-01,
          -7.9035e-

In [55]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)